<a href="https://colab.research.google.com/github/GTimothee/compare-rags/blob/main/run_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
- change runtime to GPU

## Pull the code

In [1]:
!git clone https://github.com/GTimothee/compare-rags.git

Cloning into 'compare-rags'...
remote: Enumerating objects: 316, done.
remote: Counting objects: 100% (316/316), done.
remote: Compressing objects: 100% (201/201), done.
remote: Total 316 (delta 155), reused 246 (delta 92), pack-reused 0 (from 0)
Receiving objects: 100% (316/316), 522.96 KiB | 4.55 MiB/s, done.
Resolving deltas: 100% (155/155), done.


In [2]:
%cd compare-rags/

/content/compare-rags


## Install dependencies

In [3]:
!sudo pip3 install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [4]:
# no idea why it fails when pip install -r requirements, so I had to extract the dependencies here
%pip install llama-index
%pip install lightrag-hku
%pip install neo4j neo4j-graphrag[mistralai]

%pip install llama-index-graph-stores-neo4j

%pip install llama-index-embeddings-openai
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-langchain

%pip install llama-index-llms-mistralai
%pip install llama-index-llms-langchain
%pip install llama-index-llms-openai
%pip install llama-index-llms-openai-like
%pip install llama-index-llms-huggingface
%pip install llama-index-llms-huggingface-api


%pip install pipmaster
%pip install langchain-openai
%pip install transformers
%pip install future

%pip install python-dotenv
%pip install datasets
%pip install langchain_openai

%pip install -e .

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; p

## [Optional] Create a neo4j instance
- create a demo instance on neo4j aura and add a DB
- add the credentials to your notebook

we need:
```
NEO4J_URL=
NEO4J_USERNAME=
NEO4J_PASSWORD=
NEO4J_DATABASE=
```



Try the connection:

# Experiment

## Configuration

Customize your config here

In [5]:
%pip install autoawq

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 105.2 MB/s eta 0:00:00
  Created wheel for autoawq: filename=autoawq-0.2.8-py3-none-any.whl size=108746 sha256=ca2447fec04ca93afd531cdd1ced8131215f7c5aa334d48ee037b4ec619effb6
  Stored in directory: /root/.cache/pip/wheels/fd/03/fe/99c1c678bfe8aca712186466969ed866f52feda95ae1dcd1b1
Successfully built autoawq
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.0
    Uninstalling transformers-4.50.0:
      Successfully uninstalled transformers-4.50.0


In [6]:
import yaml

model_name = "TheBloke/Mistral-7B-Instruct-v0.2-AWQ" # "HuggingFaceH4/zephyr-7b-beta"

data = {
    "framework": "llama_index",
    "description": "test installation",
    "dataset_path": "m-ric/huggingface_doc",
    "index_dirpath": "data/llama_index/",
    "n_samples": 8,
    "output_dir": "experiments/llama_index/test",
    "llm": f"huggingface_{model_name}",
    "embedding_model": "huggingface-sentence-transformers",
    "version": "test",
    "eval_dataset_path": "GTimothee/huggingface_doc_qa_10_docs"
}

with open("config.yaml", "w") as file:
  yaml.dump(data, file)


In [7]:
from pathlib import Path
Path(data['index_dirpath']).mkdir(parents=True, exist_ok=True)
Path(data['output_dir']).mkdir(parents=True, exist_ok=True)

## Run experiment

In [8]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `hf_api_key_access_any_repo` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active to

In [11]:
!python scripts/build_graph.py config.yaml

2025-03-28 13:33:17.694256: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743168797.714134    3283 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743168797.721426    3283 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
README.md: 100% 21.0/21.0 [00:00<00:00, 179kB/s]
huggingface_doc.csv: 100% 22.0M/22.0M [00:01<00:00, 17.0MB/s]
Generating train split: 100% 2647/2647 [00:00<00:00, 5675.30 examples/s]
modules.json: 100% 349/349 [00:00<00:00, 2.54MB/s]
config_sentence_transformers.json: 100% 116/116 [00:00<00:00, 885kB/s]
README.md: 100% 10.5k/10.5k [00:00<00:00, 43.8MB/s]
sentence_bert_config.json: 100% 53.0/53.0 [00:00<00:00, 421kB/s]
config.json: 1

## Save a backup of the DB
At this point you may want to download the zip version of the DB you just made, so that if there is a critical failure in the following cells, you don't lose the DB you just generated, as it takes time... (>300 seconds for 8 documents in my example with llama-index)

In [12]:
import zipfile
import os

def zip_directory(folder_path, zip_file_name):
    """Zips a directory and its contents."""
    try:
        with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, _, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, folder_path)
                    zipf.write(file_path, arcname=arcname)
        print(f"Successfully created '{zip_file_name}'")
    except FileNotFoundError:
        print(f"Error: Directory '{folder_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


zip_directory("/content/compare-rags/data/llama_index", "llama_index_graphdb.zip")


Successfully created 'llama_index_graphdb.zip'


## Running the rag

In [15]:
!python scripts/test_rag.py config.yaml "how to create a new endpoint with Huggingface?"  # took 68s

2025-03-28 13:52:31.676145: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743169951.696244    8281 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743169951.702312    8281 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Question: how to create a new endpoint with Huggingface?
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Time taken: 68.41904616355896 seconds
Answer: 
To create a new endpoint with Huggingface, follow these steps:
1. Log in to the Huggingface Endpoints platform.
2. Go to the Endp